In [1]:
array=[[1,2], [2,3], [4, 5], [5,6], [6,7], [7,8], [8,9]]
array=array[-5:]
print(array)

[[4, 5], [5, 6], [6, 7], [7, 8], [8, 9]]


In [2]:
import os
from pymongo import MongoClient

In [3]:
image_collection_path=r"D:\\personalCode\\RAG-runner-deployment\\backend\\mongodb_sample_images"

In [4]:
MONGO_URI = "mongodb://localhost:27017/"
DB_NAME = "img_label_db"
COLLECTION_NAME = "images"

In [5]:
client = MongoClient(MONGO_URI)
db = client[DB_NAME]
collection = db[COLLECTION_NAME]
print(f"Connected to MongoDB database: '{DB_NAME}'")

Connected to MongoDB database: 'img_label_db'


In [5]:
inserted_count = 0
skipped_count = 0
supported_extensions = ('.jpg', '.jpeg', '.png', '.gif', '.bmp', '.tiff', '.webp')

In [7]:
print(f"Starting scan of '{image_collection_path}'...")

for root, dirs, files in os.walk(image_collection_path):
    # Determine the label based on the subfolder name
    # If the root is the main IMAGE_COLLECTION_ROOT, there's no specific label for that level,
    # so we'll look at the immediate subfolder.
    relative_path = os.path.relpath(root, image_collection_path)
    
    # Skip the root directory itself unless it contains images directly (which is unlikely for labels)
    if relative_path == ".":
        # Check if there are images directly in the root for a generic label, or skip.
        # For this script, we assume subfolders define labels.
        print(f"Skipping root directory '{root}' as a label source.")
        continue
        
    # The label is the immediate subfolder name
    label = os.path.basename(root)

    print(f"\nProcessing folder: '{root}' (Label: '{label}')")

    for file in files:
        if file.lower().endswith(supported_extensions):
            image_name = file
            image_full_path = os.path.join(root, file)

            document = {
                "image_name": image_name,
                "image_path": image_full_path,
                "labels": [label], # Label is the subfolder name
                # No 'metadata' or 'capture_date' as per your requirements
            }

            try:
                # Optional: Check if a document with this image_path already exists to avoid duplicates
                # if collection.count_documents({"image_path": image_full_path}) == 0:
                collection.insert_one(document)
                print(f"  Inserted: {image_name} (Label: {label})")
                inserted_count += 1
                # else:
                #     print(f"  Skipped (already exists): {image_name}")
                #     skipped_count += 1
            except Exception as e:
                print(f"  Error inserting {image_name}: {e}")
                skipped_count += 1
        else:
            print(f"  Skipped (unsupported extension): {file}")
            skipped_count += 1

print("\n--- Script Finished ---")
print(f"Total images inserted: {inserted_count}")
print(f"Total files skipped: {skipped_count}")

# Close the MongoDB connection
client.close()
print("MongoDB connection closed.")

Starting scan of 'D:\\personalCode\\RAG-runner-deployment\\backend\\mongodb_sample_images'...
Skipping root directory 'D:\\personalCode\\RAG-runner-deployment\\backend\\mongodb_sample_images' as a label source.

Processing folder: 'D:\\personalCode\\RAG-runner-deployment\\backend\\mongodb_sample_images\russia' (Label: 'russia')
  Inserted: drone.png (Label: russia)
  Inserted: submarine.jpg (Label: russia)
  Inserted: tank.png (Label: russia)

--- Script Finished ---
Total images inserted: 3
Total files skipped: 0
MongoDB connection closed.


In [6]:
client=MongoClient(MONGO_URI)
db=client[DB_NAME]
collection=db[COLLECTION_NAME]

In [9]:
print(client)
print(db)
print(collection)

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)
Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'img_label_db')
Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'img_label_db'), 'images')


In [9]:
for img in collection.find():
    print(img)

{'_id': ObjectId('684d0aae89fa931ead9b2ba5'), 'image_name': 'drone.png', 'image_path': 'D:\\\\personalCode\\\\RAG-runner-deployment\\\\backend\\\\mongodb_sample_images\\russia\\drone.png', 'labels': ['russia']}
{'_id': ObjectId('684d0aae89fa931ead9b2ba6'), 'image_name': 'submarine.jpg', 'image_path': 'D:\\\\personalCode\\\\RAG-runner-deployment\\\\backend\\\\mongodb_sample_images\\russia\\submarine.jpg', 'labels': ['russia']}
{'_id': ObjectId('684d0aae89fa931ead9b2ba7'), 'image_name': 'tank.png', 'image_path': 'D:\\\\personalCode\\\\RAG-runner-deployment\\\\backend\\\\mongodb_sample_images\\russia\\tank.png', 'labels': ['russia']}
